In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
%%time
import gc
import joblib
import h5py
import pathlib
import numpy as np 


dir_path = '/store/clues/HESTIA/RE_SIMS/8192/GAL_FOR/17_11/output_2x2.5Mpc/groups_127/'

SubhMass = []
SubhPos  = []
SubhMTy  = []
SubhR50  = []

for i in range(0,8):
    
    snap = h5py.File(dir_path + 'fof_subhalo_tab_127.'+ str('%s'%i) +'.hdf5','r')

    sub_mass = snap['Subhalo/SubhaloMass'][()]
    sub_pos  = snap['Subhalo/SubhaloPos'][()]
    sub_mty  = snap['Subhalo/SubhaloMassType'][()]
    sub_r50  = snap['Subhalo/SubhaloHalfmassRad'][()]

    SubhMass.append(sub_mass)
    SubhPos.append(sub_pos)
    SubhMTy.append(sub_mty)
    SubhR50.append(sub_r50)

SubhMass = np.concatenate(SubhMass)
SubhPos  = np.concatenate(SubhPos)
SubhMTy  = np.concatenate(SubhMTy)
SubhR50  = np.concatenate(SubhR50)

gc.collect()

CPU times: user 302 ms, sys: 165 ms, total: 467 ms
Wall time: 735 ms


0

In [3]:
import numpy as np

print (np.shape(SubhPos))

sort = np.argsort(SubhMass)

SMass = SubhMass[sort][::-1]*1e10
SMTy  = SubhMTy[sort][::-1]*1e10
SR50  = SubhR50[sort][::-1]

print (SMass[:10])
print (SMTy[:10])
print (SR50[:10])

(81461, 3)
[1.5752759e+12 1.1699873e+12 5.7578933e+11 5.5521280e+11 2.3570245e+11
 1.8116307e+11 1.6131521e+11 1.4867379e+11 1.4413586e+11 1.4381996e+11]
[[1.10465729e+11 1.38343245e+12 0.00000000e+00 0.00000000e+00
  8.12707922e+10 1.06893328e+08 0.00000000e+00]
 [5.62114560e+10 1.03561803e+12 0.00000000e+00 0.00000000e+00
  7.81197230e+10 3.81372840e+07 0.00000000e+00]
 [6.23428690e+10 3.93908191e+11 0.00000000e+00 0.00000000e+00
  1.19115170e+11 4.23098592e+08 0.00000000e+00]
 [4.31543788e+10 4.82571387e+11 0.00000000e+00 0.00000000e+00
  2.94764728e+10 1.05639230e+07 0.00000000e+00]
 [2.00611615e+10 2.02057843e+11 0.00000000e+00 0.00000000e+00
  1.35750205e+10 8.42748100e+06 0.00000000e+00]
 [1.68133530e+10 1.51495737e+11 0.00000000e+00 0.00000000e+00
  1.28446607e+10 9.31647900e+06 0.00000000e+00]
 [1.71730565e+10 1.35264256e+11 0.00000000e+00 0.00000000e+00
  8.87177728e+09 6.11738700e+06 0.00000000e+00]
 [1.81018071e+10 1.24038636e+11 0.00000000e+00 0.00000000e+00
  6.53002496e+

In [4]:
H_pos = SubhPos[sort][::-1]
print (H_pos[:10])

[[48.81521  46.704514 53.605064]
 [48.71576  47.06235  53.33718 ]
 [46.974728 49.455475 51.771206]
 [47.528534 46.364803 54.832882]
 [47.325207 47.259197 51.91327 ]
 [47.791496 47.55743  53.327   ]
 [47.41012  48.306732 52.663097]
 [49.867104 47.25419  52.225414]
 [45.62381  46.928238 54.103687]
 [48.525105 48.03401  51.99552 ]]


In [5]:
%%time
import gc
import joblib
import h5py
import pathlib
import numpy as np

dir_path = '/store/clues/HESTIA/RE_SIMS/8192/GAL_FOR/17_11/output_2x2.5Mpc/snapdir_127/'

StrPos  = []
StrMass = []

GasPos  = []
GasMass = []

DrkPos  = []
DrkMass = []


for i in range(0,8):
    snap = h5py.File(dir_path + 'snapshot_127.'+ str('%s'%i) +'.hdf5','r')
    
    #stars
    pos_str    = snap['PartType4/Coordinates'][()]
    mass_str   = snap['PartType4/Masses'][()]
   
    StrPos.append(pos_str)
    StrMass.append(mass_str)

    #gas
    pos_gas    = snap['PartType0/Coordinates'][()]
    mass_gas   = snap['PartType0/Masses'][()]
    
    GasPos.append(pos_gas)
    GasMass.append(mass_gas)

    #dark matter
    pos_drk    = snap['PartType1/Coordinates'][()]
    mass_drk   = snap['PartType1/Masses'][()]
    
    DrkPos.append(pos_drk)
    DrkMass.append(mass_drk)

    
a = snap.get('Header').attrs['Time']
h = snap.get('Header').attrs['HubbleParam']

Omega0   = snap.get('Header').attrs['Omega0']
Omegab   = snap.get('Header').attrs['OmegaBaryon']
OmegaL   = snap.get('Header').attrs['OmegaLambda']
Redshift = snap.get('Header').attrs['Redshift']

Unit_Length = snap.get('Header').attrs['UnitLength_in_cm']
Unit_Mass   = snap.get('Header').attrs['UnitMass_in_g']
Unit_Veloc  = snap.get('Header').attrs['UnitVelocity_in_cm_per_s']

StrPos  = np.concatenate(StrPos)
StrMass = np.concatenate(StrMass)

GasPos  = np.concatenate(GasPos)
GasMass = np.concatenate(GasMass)

DrkPos  = np.concatenate(DrkPos)
DrkMass = np.concatenate(DrkMass)

gc.collect()

CPU times: user 1min 31s, sys: 1min 44s, total: 3min 16s
Wall time: 31.4 s


0

In [6]:
import numpy as np
subh = np.loadtxt('../_data/subhalos.dat')

halo_mask, = np.where((subh[:,15]==0) & (subh[:,16]==0))

subh_name = subh[:,0][halo_mask]
subh_ind  = np.int_(subh_name)
# nsubh = len(subh_name)

print(subh_ind)

[ 0  1  3  4  5  6  7  9 20 21 22 27 28 31 33 35 36 37 38 39 41 42 45 46
 49 50 51 53 54 64 65 66 69 73 74 79 84 85 87 88 90 91 92 95 96]


In [11]:
%%time

import numpy as np
import gc

G = 4.299e-6 #kpc*(km/s)*Msol^-1

H0 = 100*h #km/s/Mpc

# omega_matter = results[0]['Omega0']
# omega_lambda = results[0]['OmegaL']
# omega0 = omega_matter + omega_lambda

# z = results[0]['Redshift']

for i in subh_ind[:8]:
    
    SPos = (StrPos - H_pos[i,:])*1000
    xstr = SPos[:,0]*a/h
    ystr = SPos[:,1]*a/h
    zstr = SPos[:,2]*a/h
    rstr = np.sqrt(xstr**2 + ystr**2 + zstr**2)
    mstr = StrMass*1e10/h
    
    GPos = (GasPos - H_pos[i,:])*1000
    xgas = GPos[:,0]*a/h
    ygas = GPos[:,1]*a/h
    zgas = GPos[:,2]*a/h
    rgas = np.sqrt(xgas**2 + ygas**2 + zgas**2)
    mgas = GasMass*1e10/h
    
    DPos = (DrkPos - H_pos[i,:])*1000
    xdrk = DPos[:,0]*a/h
    ydrk = DPos[:,1]*a/h
    zdrk = DPos[:,2]*a/h
    rdrk = np.sqrt(xdrk**2 + ydrk**2 + zdrk**2)
    mdrk = DrkMass*1e10/h
    
    st, = np.where(rstr < 1000)
    dm, = np.where(rdrk < 1000)
    gs, = np.where(rstr < 1000)
    
    r = np.concatenate([rstr[st], rgas[gs], rdrk[dm]])
    m = np.concatenate([mstr[st], mgas[gs], mdrk[dm]])

    psort = np.argsort(r)
    
    
    vol  = 4./3 * np.pi * r[psort]**3
    mass = np.cumsum(m[psort])
    rho  = mass / vol

    H0_kpc = H0/1000 #paso a kpc
    
    rho_crit = 3*H0_kpc**2 / (8*np.pi*G)

    mask, = np.where(rho > 200*rho_crit)
    
    Mvir = mass[mask][-1]
    Rvir = r[psort][mask][-1] 
    
#     print (i)
    print ('subhalo', str('%03d'%i), 'Mvir = ', str('%.4E'%Mvir), 'Rvir = ', str('%4.8f'%Rvir))
    
    gc.collect()
    

subhalo 000 Mvir =  2.6059E+12 Rvir =  290.01156544
subhalo 001 Mvir =  2.2134E+12 Rvir =  274.65235988
subhalo 003 Mvir =  7.9338E+11 Rvir =  195.10067109
subhalo 004 Mvir =  3.3978E+11 Rvir =  147.06049023
subhalo 005 Mvir =  2.6794E+11 Rvir =  135.86501130
subhalo 006 Mvir =  2.4268E+11 Rvir =  131.45412989
subhalo 007 Mvir =  2.0014E+11 Rvir =  123.27589266
subhalo 009 Mvir =  1.9425E+11 Rvir =  122.05354973
CPU times: user 26min 29s, sys: 9min 8s, total: 35min 37s
Wall time: 7min 48s


In [69]:
msort = np.argsort(Mvir)
shID = msort

data = np.ndarray([nsub, 3])
data[:,0] = Mvir[msort][::-1]
data[:,1] = Rvir[msort][::-1]
data[:,2] = shID[::-1]

np.savetxt('../_data/Mvir_Rvir.dat', data, fmt = ('%.5E','%12.8f', '%03d'))

In [59]:
for i in range(11):
    print ('%.3E'%Mvir[i])

1.959E+12
1.833E+11
1.568E+12
1.188E+11
2.816E+10
1.538E+11
1.912E+12
1.796E+12
3.358E+09
1.845E+10
3.872E+09


In [50]:
msort = np.argsort(Mvir)
print (msort[::-1])

[ 0  6  7  2  1  5  3  4  9 10  8 39 31 32 33 34 35 36 37 38 44 40 41 42
 43 29 45 46 47 30 24 28 18 11 12 13 14 15 16 17 19 27 20 21 22 23 98 25
 26 48 99 50 75 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 76 74 51 73 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69
 70 71 72 49]


In [17]:
(1.88e-29)*h**2 *(3.086e21)**3 / (1.989e33) 

127.58094145367797

In [39]:
print (rho_crit*200)

25504.62350081994


In [41]:
print (r[psort][mask])

[  259.55519224   259.58658917   259.59027663 ... 90929.78325109
 91228.94830954 91241.25829807]


In [ ]:
for i in range(100):
    print (SMASS[i], Mvir[i])

In [ ]:

r = np.concatenate(rstr,rgas,rdrk)
m = np.concatenate(mstr,mgas,mdrk)

psort = np.argsort(r)

vol  = 4./3 * np.pi * r[psort]**3
mass = m[psort]
rho  = mass / vol

rho_crit = 3*H**2 / (8*np.pi*G)

mask, = np.where(rho < 200*rho_crit)
Mvir = mvir[-1]

In [ ]:
h = 0.6777
G = 4.299e-6 #kpc*(km/s)*Msol^-1
a0 = 1
H0 = h*100
omega_lambda = 0.682
omega_matter = 0.318
omega0 = omega_matter + omega_lambda

z = a0/aexp[isnap] - 1.
Ht = H0*np.sqrt(omega_lambda+(1-omega0)*(1+z)**2+omega_matter*(1+z)**3)
time = tiempo.conv(z, h, omega_lambda, omega_matter)

#---aca paso las coordenadas respecto al centro de la galaxia------
xstr = (pstr[:,0]-cm[0])*aexp[isnap]/h
ystr = (pstr[:,1]-cm[1])*aexp[isnap]/h
zstr = (pstr[:,2]-cm[2])*aexp[isnap]/h

v_x = vel[:,0] *np.sqrt(aexp[isnap]) + Ht *xstr/1000.
v_y = vel[:,1] *np.sqrt(aexp[isnap]) + Ht *ystr/1000.
v_z = vel[:,2] *np.sqrt(aexp[isnap]) + Ht *zstr/1000.